In [1]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langdetect import detect
from sentence_transformers import SentenceTransformer, util


In [2]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
sentence_transformer_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [3]:
from langdetect import detect

In [4]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

def get_similar_answer(user_input, df, domain):
    user_embedding = sentence_transformer_model.encode([user_input], convert_to_tensor=True)
    max_similarity = -1
    most_similar_question = None

    for index, row in df.iterrows():
        question = row['سوالات']
        question_domain = row['Domain']  
        if domain is not None and domain != question_domain:
            continue  

        question_embedding = sentence_transformer_model.encode([question], convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(user_embedding, question_embedding)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_question = question

    if most_similar_question:
        answer = df[df['سوالات'] == most_similar_question]['جوابات'].iloc[0]
        return answer
    else:
        return "معاف کریں، میں اس سوال کا جواب نہیں دے سکتا۔"




In [5]:
def get_domain(user_input):
    # Define a set of domain keywords
    domain_keywords = {
        'نئے': 'نئے',
        'پولیس': 'پولیس ویریفکیشن',
        'کریکٹر': 'کریکٹر سرٹیفکیٹ',
        'ملازم': 'ملازم',
        'ایف آئی آر': 'ایف آئی آر',
        'جرم': 'جرم',
        'گمشدگی': 'گمشدگی',
        'گاڑی': 'گاڑی',
        'کرایہ دار': 'کرایہ دار',
        'لرنر ڈرائیونگ': 'لرنر ڈرائیونگ',
        'ڈپلیکیٹ ڈرائیونگ لائسنس': 'ڈپلیکیٹ ڈرائیونگ لائسنس',
        'رینیو': 'رینیو',
        'نقل': 'نقل',
        'انٹرنیشنل ڈرائیونگ لائسنس': 'انٹرنیشنل ڈرائیونگ لائسنس',
        'خوش آمدید': 'خوش آمدید',
        'خواتین': 'خواتین',
        # Add more keywords for other domains if needed
    }

    user_input_lower = user_input.lower()

    detected_domain = None
    for keyword, domain in domain_keywords.items():
        if keyword in user_input_lower:
            detected_domain = domain
            break

    return detected_domain


In [ ]:
def chat_with_user(df):
    print("چیٹ بوٹ: میں چیٹ جی پی ٹی ہوں، آپ کا دوستانہ چیٹ بوٹ۔")

    domain = None
    dialogue_history = []

    while True:
        user_input = input("آپ: ")

        user_language = detect(user_input)

        if user_language != 'ur':
            print("چیٹ بوٹ: معاف کریں، صرف اردو میں جواب دیں۔")
            continue  

        if user_input.lower() in ["bye", "exit", "الوداع", "خدا حافظ"]:
            print("چیٹ بوٹ: دنیا کو آپ کا خدا حافظ کہہ رہا ہے۔")
            break

        new_domain = get_domain(user_input)

        if new_domain:
            domain = new_domain

        response = get_similar_answer(user_input, df, domain)
        print("چیٹ بوٹ:", response)

        dialogue_history.append((user_input, response, domain))

        if domain:
            print("چیٹ بوٹ: یہ سوال خصوصیت '{}' کے تحت ہے۔".format(domain))

if __name__ == "__main__":
    file_path = 'dataset1.xlsx'
    try:
        df = pd.read_excel(file_path)
        chat_with_user(df)
    except FileNotFoundError:
        print("معاف کریں، ڈیٹا فائل (dataset.xlsx) نہیں ملا۔")
    except Exception as e:
        print("کچھ خرابی ہو گئی، براہ کرم دوبارہ کوشش کریں۔")
        print("خرابی کی تفصیل: ", e)


چیٹ بوٹ: میں چیٹ جی پی ٹی ہوں، آپ کا دوستانہ چیٹ بوٹ۔
آپ: خواتین پر تشدد کی رپورٹ درج کرانے کے لیے کون کون سی دستاویزات مطلوب ہیں؟
چیٹ بوٹ: ضروری دستاویزات:

    درخواست دہندہ کا اصلی شناختی کارڈ اور 2 کاپیاں
    درخواست دہندہ کی تحریری درخواست اگر موجود ہے
چیٹ بوٹ: یہ سوال خصوصیت 'خواتین' کے تحت ہے۔
